# Load the modules

In [27]:
import cudf
from collections import OrderedDict
import numpy as np
import datetime as dt

from dask_cuda import LocalCUDACluster
from dask.distributed import Client

from blazingsql import BlazingContext

import dask_cudf

In [2]:
print(cudf.__version__)

0.16.0


In [3]:
cluster = LocalCUDACluster()
client = Client(cluster)
client.restart()

Client Scheduler: tcp://127.0.0.1:36357 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 67.48 GB


# Read the data

In [3]:
!head -n 10 ../../data/parking_MayJun2019.csv

head: cannot open '../../data/parking_MayJun2019.csv' for reading: No such file or directory


In [31]:
transactions_path = '../../data/seattle_parking/parking_MayJun2019.parquet/partition_idx={partition}/*'
transactions_parq = [transactions_path.format(partition=p) for p in range(transactions_partitions_cnt)]

In [31]:
parking_transactions = dask_cudf.read_parquet(
    transactions_parq
)

parking_locations = dask_cudf.read_parquet('../../data/seattle_parking/parking_locations.parquet/')

Features

1. Occupancy at T minus 15mins, 1h, 3h, 1d, 2d, 3d, 7d, 14d
2. PaidParkingArea & SubArea

In [28]:
bc = BlazingContext(dask_client=client)

BlazingContext ready


In [32]:
bc.create_table('parking_transactions', parking_transactions)

In [51]:
bc.sql('SELECT * FROM parking_transactions LIMIT 10').compute()

,OccupancyDateTime,PaidOccupancy,SourceElementKey,dow
0,2019-05-24 08:30:00,1,31493,4
1,2019-05-24 13:41:00,8,76198,4
2,2019-05-24 08:33:00,3,34505,4
3,2019-05-24 10:19:00,4,13101,4
4,2019-05-24 15:05:00,1,37189,4
5,2019-05-24 14:11:00,3,9721,4
6,2019-06-10 17:16:00,4,52530,0
7,2019-06-10 19:12:00,9,60005,0
8,2019-06-10 08:46:00,4,60234,0
9,2019-06-10 17:53:00,3,10213,0


In [54]:
bc.sql('''
    SELECT SourceElementKey 
        , OccupancyDateTime 
        , dow AS DOW
        , HOUR(OccupancyDateTime) - 1 AS hr_minus_1
        , HOUR(OccupancyDateTime) - 3 AS hr_minus_3
        , DAYOFMONTH(OccupancyDateTime) - 1 AS day_minus_1
    FROM parking_transactions 
    ORDER BY SourceElementKey
        , OccupancyDateTime
    LIMIT 10
''').compute()

,SourceElementKey,OccupancyDateTime,DOW,hr_minus_1,hr_minus_3,day_minus_1
0,1001,2019-05-01 09:00:00,2,8,6,0
1,1001,2019-05-01 09:01:00,2,8,6,0
2,1001,2019-05-01 09:02:00,2,8,6,0
3,1001,2019-05-01 09:03:00,2,8,6,0
4,1001,2019-05-01 09:04:00,2,8,6,0
5,1001,2019-05-01 09:05:00,2,8,6,0
6,1001,2019-05-01 09:06:00,2,8,6,0
7,1001,2019-05-01 09:07:00,2,8,6,0
8,1001,2019-05-01 09:08:00,2,8,6,0
9,1001,2019-05-01 09:09:00,2,8,6,0


In [25]:
df_locations['ParkingArea_concat'] = (
    df_locations['PaidParkingArea'].str.replace(' ', '_')
    .str.cat(df_locations['PaidParkingSubArea'], sep='__')
)

df_locations.head()

,SourceElementKey,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,ParkingCategory,Location,ParkingArea_concat
0,1009,1ST AVE BETWEEN MADISON ST AND SPRING ST,SW,30,5,Commercial Core,Waterfront,Paid Parking,POINT (-122.3366575 47.60501765),Commercial_Core__Waterfront
1,1018,1ST AVE BETWEEN STEWART ST AND VIRGINIA ST,NE,120,12,Belltown,South,Paid Parking,POINT (-122.34188878 47.61073498),Belltown__South
2,1045,1ST AVE N BETWEEN REPUBLICAN ST AND MERCER ST,W,120,11,Uptown,Core,Paid Parking,POINT (-122.35549857 47.62391565),Uptown__Core
3,1213,1ST AVE S BETWEEN YESLER WAY AND S WASHINGTON ST,W,120,11,Pioneer Square,Core,Paid Parking,POINT (-122.33428548 47.60129336),Pioneer_Square__Core
4,1214,1ST AVE S BETWEEN YESLER WAY AND S WASHINGTON ST,E,120,11,Pioneer Square,Core,Paid Parking,POINT (-122.3340747 47.60129576),Pioneer_Square__Core
